In [23]:
import pandas as pd 
import numpy as np 
from prediction_utils import get_5_game_team_standing, get_sched_strength, knapsack_fpl_xi

# Update Raw GW Data and Set Param!!! 

Update `data/2023-24/gws/merged_gw.csv` by running the following in terminal: 
1. `git fetch upstream master`
2. `git checkout upstream/master -- data/2023-24/gws/merged_gw.csv`

In [24]:
current_gw = 5

# Import Data

In [25]:
df = pd.read_csv("../../data/2023-24/gws/merged_gw.csv")
df.head()

,name,position,team,xP,assists,bonus,bps,clean_sheets,creativity,element,...,team_h_score,threat,total_points,transfers_balance,transfers_in,transfers_out,value,was_home,yellow_cards,GW
0,Femi Seriki,DEF,Sheffield Utd,0.5,0,0,0,0,0.0,653,...,0,0.0,0,0,0,0,40,True,0,1
1,Jack Hinshelwood,MID,Brighton,1.5,0,0,0,0,0.0,621,...,4,0.0,0,0,0,0,45,True,0,1
2,Jadon Sancho,MID,Man Utd,3.0,0,0,4,0,11.3,397,...,1,8.0,1,0,0,0,70,True,0,1
3,Rhys Norrington-Davies,DEF,Sheffield Utd,0.1,0,0,0,0,0.0,487,...,0,0.0,0,0,0,0,40,True,0,1
4,Vitaly Janelt,MID,Brentford,2.1,0,0,6,0,11.5,105,...,2,17.0,2,0,0,0,55,True,0,1


# Get relevant columns

In [26]:
df_cleaned = df[[
    "GW", 
    "name", 
    "position", 
    "team", 
    "minutes", 
    "goals_scored", 
    "assists", 
    "clean_sheets", 
    "saves", 
    "penalties_saved", 
    "penalties_missed", 
    "bonus", 
    "goals_conceded", 
    "yellow_cards", 
    "red_cards", 
    "own_goals", 
    "expected_goals", 
    "expected_assists", 
    "expected_goal_involvements", 
    "expected_goals_conceded"
]]

df_cleaned.head()

,GW,name,position,team,minutes,goals_scored,assists,clean_sheets,saves,penalties_saved,penalties_missed,bonus,goals_conceded,yellow_cards,red_cards,own_goals,expected_goals,expected_assists,expected_goal_involvements,expected_goals_conceded
0,1,Femi Seriki,DEF,Sheffield Utd,0,0,0,0,0,0,0,0,0,0,0,0,0.00,0.00,0.00,0.00
1,1,Jack Hinshelwood,MID,Brighton,0,0,0,0,0,0,0,0,0,0,0,0,0.00,0.00,0.00,0.00
2,1,Jadon Sancho,MID,Man Utd,22,0,0,0,0,0,0,0,0,0,0,0,0.00,0.05,0.05,1.08
3,1,Rhys Norrington-Davies,DEF,Sheffield Utd,0,0,0,0,0,0,0,0,0,0,0,0,0.00,0.00,0.00,0.00
4,1,Vitaly Janelt,MID,Brentford,90,0,0,0,0,0,0,0,2,0,0,0,0.02,0.01,0.03,1.26


# Group by recent 5 games

In [27]:
df_period = df_cleaned[df_cleaned["GW"].isin(range(current_gw-5, current_gw))]

df_grouped = df_period.groupby(["name", "position"]).agg({
    "team": "max", 
    "GW": "max", 
    "minutes": "mean", 
    "goals_scored": "mean", 
    "assists": "mean", 
    "clean_sheets": "mean", 
    "saves": "mean", 
    "penalties_saved": "mean", 
    "penalties_missed": "mean", 
    "bonus": "mean", 
    "goals_conceded": "mean", 
    "yellow_cards": "mean", 
    "red_cards": "mean", 
    "own_goals": "mean", 
    "expected_goals": "mean", 
    "expected_assists": "mean", 
    "expected_goal_involvements": "mean", 
    "expected_goals_conceded": "mean"
}).reset_index()

df_grouped.head()

,name,position,team,GW,minutes,goals_scored,assists,clean_sheets,saves,penalties_saved,penalties_missed,bonus,goals_conceded,yellow_cards,red_cards,own_goals,expected_goals,expected_assists,expected_goal_involvements,expected_goals_conceded
0,Aaron Connolly,FWD,Brighton,4,0.00,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0000,0.0000,0.0000,0.0000
1,Aaron Cresswell,DEF,West Ham,4,0.00,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0000,0.0000,0.0000,0.0000
2,Aaron Hickey,DEF,Brentford,4,79.75,0.0,0.0,0.25,0.00,0.0,0.0,0.0,1.25,0.5,0.0,0.0,0.0175,0.0300,0.0475,1.0500
3,Aaron Ramsdale,GK,Arsenal,4,90.00,0.0,0.0,0.25,1.25,0.0,0.0,0.0,1.00,0.0,0.0,0.0,0.0000,0.0025,0.0025,0.9325
4,Aaron Ramsey,MID,Burnley,4,11.00,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0000,0.0050,0.0050,0.1500


In [28]:
df_grouped.head()

,name,position,team,GW,minutes,goals_scored,assists,clean_sheets,saves,penalties_saved,penalties_missed,bonus,goals_conceded,yellow_cards,red_cards,own_goals,expected_goals,expected_assists,expected_goal_involvements,expected_goals_conceded
0,Aaron Connolly,FWD,Brighton,4,0.00,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0000,0.0000,0.0000,0.0000
1,Aaron Cresswell,DEF,West Ham,4,0.00,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0000,0.0000,0.0000,0.0000
2,Aaron Hickey,DEF,Brentford,4,79.75,0.0,0.0,0.25,0.00,0.0,0.0,0.0,1.25,0.5,0.0,0.0,0.0175,0.0300,0.0475,1.0500
3,Aaron Ramsdale,GK,Arsenal,4,90.00,0.0,0.0,0.25,1.25,0.0,0.0,0.0,1.00,0.0,0.0,0.0,0.0000,0.0025,0.0025,0.9325
4,Aaron Ramsey,MID,Burnley,4,11.00,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0000,0.0050,0.0050,0.1500


# Add Schedule Strength

## Pre-process fixtures data

Update `data/2023-24/fixtures.csv` by running the following in terminal: 
1. `git fetch upstream master`
2. `git checkout upstream/master -- data/2023-24/fixtures.csv`

In [29]:
df_fixtures = pd.read_csv("../../data/2023-24/fixtures.csv")
df_teams = pd.read_csv("../../data/2023-24/teams.csv")
df_standing = pd.read_csv("../../data/2023-24/standing.csv")

In [30]:
df_fixtures["event"].fillna(0, inplace=True)
df_fixtures["event"] = df_fixtures["event"].astype("int")
df_merged_home = pd.merge(df_fixtures[["event", "team_h", "team_a"]], df_teams[["name", "id"]], how="left", left_on="team_h", right_on="id")
df_merged_away = pd.merge(df_merged_home, df_teams[["name", "id"]], how="left", left_on="team_a", right_on="id")
df_fixtures_cleaned = df_merged_away[["event", "name_x", "name_y"]]
df_fixtures_cleaned.columns = ["GW", "home_team", "away_team"]
df_fixtures_cleaned.head()

,GW,home_team,away_team
0,0,Luton,Burnley
1,0,Man City,Brentford
2,1,Burnley,Man City
3,1,Arsenal,Nott'm Forest
4,1,Bournemouth,West Ham


## Add team standing

In [31]:
df_grouped["team_standing"] = df_grouped.apply(lambda row: get_5_game_team_standing(row["team"], current_gw, df_standing), axis=1)
df_grouped.head()

,name,position,team,GW,minutes,goals_scored,assists,clean_sheets,saves,penalties_saved,...,bonus,goals_conceded,yellow_cards,red_cards,own_goals,expected_goals,expected_assists,expected_goal_involvements,expected_goals_conceded,team_standing
0,Aaron Connolly,FWD,Brighton,4,0.00,0.0,0.0,0.00,0.00,0.0,...,0.0,0.00,0.0,0.0,0.0,0.0000,0.0000,0.0000,0.0000,3.75
1,Aaron Cresswell,DEF,West Ham,4,0.00,0.0,0.0,0.00,0.00,0.0,...,0.0,0.00,0.0,0.0,0.0,0.0000,0.0000,0.0000,0.0000,5.25
2,Aaron Hickey,DEF,Brentford,4,79.75,0.0,0.0,0.25,0.00,0.0,...,0.0,1.25,0.5,0.0,0.0,0.0175,0.0300,0.0475,1.0500,7.25
3,Aaron Ramsdale,GK,Arsenal,4,90.00,0.0,0.0,0.25,1.25,0.0,...,0.0,1.00,0.0,0.0,0.0,0.0000,0.0025,0.0025,0.9325,4.25
4,Aaron Ramsey,MID,Burnley,4,11.00,0.0,0.0,0.00,0.00,0.0,...,0.0,0.00,0.0,0.0,0.0,0.0000,0.0050,0.0050,0.1500,18.75


## Calculate 5-game schedule strength

In [32]:
df_grouped["sched_strength"] = df_grouped.apply(lambda row: get_sched_strength(row["team"], current_gw, df_fixtures_cleaned, df_standing), axis=1)
df_grouped.head()

,name,position,team,GW,minutes,goals_scored,assists,clean_sheets,saves,penalties_saved,...,goals_conceded,yellow_cards,red_cards,own_goals,expected_goals,expected_assists,expected_goal_involvements,expected_goals_conceded,team_standing,sched_strength
0,Aaron Connolly,FWD,Brighton,4,0.00,0.0,0.0,0.00,0.00,0.0,...,0.00,0.0,0.0,0.0,0.0000,0.0000,0.0000,0.0000,3.75,8.2
1,Aaron Cresswell,DEF,West Ham,4,0.00,0.0,0.0,0.00,0.00,0.0,...,0.00,0.0,0.0,0.0,0.0000,0.0000,0.0000,0.0000,5.25,9.0
2,Aaron Hickey,DEF,Brentford,4,79.75,0.0,0.0,0.25,0.00,0.0,...,1.25,0.5,0.0,0.0,0.0175,0.0300,0.0475,1.0500,7.25,14.4
3,Aaron Ramsdale,GK,Arsenal,4,90.00,0.0,0.0,0.25,1.25,0.0,...,1.00,0.0,0.0,0.0,0.0000,0.0025,0.0025,0.9325,4.25,9.8
4,Aaron Ramsey,MID,Burnley,4,11.00,0.0,0.0,0.00,0.00,0.0,...,0.00,0.0,0.0,0.0,0.0000,0.0050,0.0050,0.1500,18.75,10.8


# Export cleaned data to csv

In [33]:
df_grouped.to_csv("../../data/cleaned_data/2024_cleaned_prediction_data.csv", index=False)